In [5]:
import tkinter as tk
from tkinter import *
from tkinter import messagebox
from PIL import Image, ImageTk

In [16]:
window = tk.Tk()
window.title("Handwritten Digit Recognition Project")
window.resizable(0,0)
load1 = Image.open("bgImage.png")
photo1 = ImageTk.PhotoImage(load1)
header = tk.Button(window, width=800, height=115, bg="white", image=photo1)
header.place(x = 0, y = 0)
canvas1 = Canvas(window, width=500, height=155, bg = "ivory")
canvas1.place(x = 5, y = 120)

def screen_capture():
    import pyscreenshot as ImageGrab
    import time
    import os
    os.startfile("C:/Windows/system32/mspaint")
    s1 = t1.get()
    os.chdir("C:/Jupyter notebook Alekya/GUI HandwrittenDigitRecognition/captured_images")
    os.mkdir(s1)
    os.chdir("C:/Jupyter notebook Alekya/GUI HandwrittenDigitRecognition/")
    images_folder = "captured_images/"+s1+"/"
    time.sleep(15)
    for i in range(0, 5):
        time.sleep(15)
        im = ImageGrab.grab(bbox=(60, 170, 400, 550)) #x1, y1, x2, y2
        print("Saved......", i) 
        im.save(images_folder + str(i) + '.png')
        print("Clear screen now and redraw now........")
    messagebox.showinfo("Result", "Capturing screen is completed!!")

def generate_dataset():
    import cv2
    import csv
    import glob

    header  =["label"]
    for i in range(0,784):
        header.append("pixel"+str(i))
    with open('dataset.csv', 'a') as f:
        writer = csv.writer(f)
        writer.writerow(header)

    for label in range(10):
        dirList = glob.glob("captured_images/"+str(label)+"/*.png")

        for img_path in dirList:
            im= cv2.imread(img_path)
            im_gray = cv2.cvtColor(im,cv2.COLOR_BGR2GRAY)
            im_gray = cv2.GaussianBlur(im_gray,(15,15), 0)
            roi= cv2.resize(im_gray,(28,28), interpolation=cv2.INTER_AREA)

            data=[]
            data.append(label)
            rows, cols = roi.shape

            
            for i in range(rows):
                for j in range(cols):
                    k =roi[i,j]
                    if k>100:
                        k=1
                    else:
                        k=0
                    data.append(k)
            with open('dataset.csv', 'a') as f:
                writer = csv.writer(f)
                writer.writerow(data)
    messagebox.showinfo("Result","Generating dataset is completed!!")

def train_save_accuracy():
    import pandas as pd
    from sklearn.utils import shuffle
    data  =pd.read_csv('dataset.csv')
    data=shuffle(data)
    X = data.drop(["label"],axis=1)
    Y= data["label"]
    from sklearn.model_selection import train_test_split
    train_x,test_x,train_y,test_y = train_test_split(X,Y, test_size = 0.2)
    import joblib
    from sklearn.svm import SVC
    classifier=SVC(kernel="linear", random_state=6)
    classifier.fit(train_x,train_y)
    joblib.dump(classifier, "model/digit_recognizer")
    from sklearn import metrics
    prediction = classifier.predict(test_x)
    acc = metrics.accuracy_score(prediction, test_y)
    acc_percent = acc * 100
    messagebox.showinfo("Result", f"Your accuracy is {acc}\n  {acc_percent}%")

def prediction():
    import joblib
    import cv2
    import numpy as np
    import time
    import pyscreenshot as ImageGrab
    model=joblib.load("model/digit_recognizer")
    img=ImageGrab.grab(bbox=(300,350,500,720))
    img.save("predict.png")
    
    im = cv2.imread("predict.png")
    load = Image.open("predict.png")
    load = load.resize((280, 280))
    photo = ImageTk.PhotoImage(load)
    img = Label(canvas3, image=photo, width=280, height=280)
    img.image = photo
    img.place(x = 0, y = 0)
    im_gray = cv2.cvtColor(im,cv2.COLOR_BGR2GRAY)
    im_gray  =cv2.GaussianBlur(im_gray, (15,15), 0)

        
    ret, im_th = cv2.threshold(im_gray,100, 255, cv2.THRESH_BINARY)
    roi = cv2.resize(im_th, (28,28), interpolation  =cv2.INTER_AREA)
    rows,cols=roi.shape

    X = []

        
    for i in range(rows):
        for j in range(cols):
            k = roi[i,j]
            if k>100:
                k=1
            else:
                k=0
            X.append(k)

    predictions=model.predict([X])
    a1 = tk.Label(canvas3,text="Prediction: ", font = ("Arial Black", 20))
    a1.place(x = 5, y = 350)
    b1 = tk.Label(canvas3,text=predictions[0], font = ("Arial Black", 20))
    b1.place(x = 200, y = 350)

b4 = tk.Button(canvas1, text = "Recognize", font = ('Arial Black', 12), bg = "white", fg = "blue", command = prediction)
b4.place(x = 50, y = 50)

canvas2 = Canvas(window, width=500, height=370, bg = "black")
canvas2.place(x = 5, y = 280)
def draw_lines(event):
    global lastx,lasty
    x,y = event.x,event.y
    canvas2.create_line((lastx,lasty,x,y),width=30,fill='white',capstyle=ROUND,smooth=TRUE,splinesteps=12)
    lastx,lasty = x,y
def activate_event(event):
    global lastx,lasty
    canvas2.bind('<B1-Motion>',draw_lines)
    lastx,lasty = event.x,event.y
canvas2.bind('<1>', activate_event)
def clear():
    canvas2.delete("all")

btn = tk.Button(canvas1, text = "Clear", font = ('Arial black', 12), fg = "black", bg = "white", command=clear)
btn.place(x = 300, y = 50)
instructions = tk.Label(canvas1, text = " Please draw any digit on the black screen and click on Recognize. ", font=('Arial Black', 10))
instructions.place(x = 5, y = 0)

canvas3 = Canvas(window, width=300, height=530, bg = "green")
canvas3.place(x = 500, y = 120)
window.geometry("800x680")
window.mainloop()